# Introduction
The project is split into three files: knn.py, cv.py and agaricus_predict.py. The file knn.py contains the k-Nearest Neighbor algorithm. The file cv.py contains the k-Fold Cross-Validation algorithm, and two scoring measures. The file agaricus_predict.py preprocesses a data set and uses the aforementioned algorithms for prediction. The files are listed in full in the appendix portion of the document. They require Python 3 and numpy to run, and for plotting matplotlib is also required.


# KNN implementation
The k-Nearest Neighbor algorithm (KNN) was implemented for classification and regression tasks. The implementation uses numpy for numerical calculations. The public interface was partly mimicked scikit-learn (only the signatures, no code). This was done so that the implementation can be compared easily, and it's already intuitive to use.


## The prediction method
```python
    def predict(self, X):
        '''
        Returns the KNN prediction for design matrix X.
        :param X: the design matrix X
        :return: the predicted values for rows of X
        '''
        dist = self.__get_distance_matrix(X)
        
        ranks = np.apply_along_axis(KNN.__rank, 1, dist)
        # Using np.nan to denote values that are not nearest neighbors
        indicators = np.where(ranks < self.n_neighbors, 1, np.nan)

        y = np.multiply(indicators, self.y)

        # Construct new matrix without nan values
        newshape = (np.shape(X)[0], self.n_neighbors)
        y = (np.reshape(y[~np.isnan(y)], newshape))

        prediction = np.apply_along_axis(self.prediction_func, 1, y)
        return prediction
```
The prediction method takes a design matrix X as parameter. First it computes a 2d-distance matrix between all pairs of rows between the given matrix and the fitted matrix. 

Then for each row, ranks of the values are calculated. The smallest distance receives the smallest rank (0), next one 1 etc. This is done so that in the future, weighting schemes can take advantage of the ranks. An indicator matrix is constructed which specifies which ranks are in the given neighborhood. The values not in the neighborhood are specified with NaN-values, because they must be differentiated with 0-values. Then a class-matrix is constructed, where the values outside the neighborhood are removed. Finally the prediction function is applied to each row. 

``` python
    def __get_distance_matrix(self, X):
        X = X.astype(np.float64)
        # Bind self to apply_metric, needed to bind self.X
        apply_metric = lambda y : self.metric(y, self.X)
        dist = np.apply_along_axis(apply_metric, 1, X)
        return dist
```
Constructing the distance matrix fast was important. The naive implementation of calculating the paired distances in for-loops took minutes to run for the used dataset. The implementation was changed rather easily to much faster (but by no means not optimal) semi-vectorized form. In the used approach, the lambda expression apply_metric calls the metric for a single vector y and the original fitted matrix X. The metric should calculate the paired distances for these, which can usually be done in matrix form. Then this lambda expression is applied to each vector in the prediction matrix X. In the end we end up calculating the distances for all pairs between the fitted X and the given X.

For future optimization, at least the regression prediction could be vectorized to matrix form. Also, it may be faster to vectorize most of the apply_along_axis calls, which are not much faster than for-loops.

# k-Fold cross-validation implementation

```python
    def split(self, X):
        minfoldsize = len(X)//self.n_splits
        rem = len(X)-minfoldsize*self.n_splits #remainder, when the splits are not even
        foldsizes = np.repeat(minfoldsize, self.n_splits)
        foldsizes[:rem] += 1 #remainders are evenly added to first rem folds
        endindices = np.cumsum(foldsizes)

        for i, (size, endindex) in enumerate(zip(foldsizes, endindices)):
            startindex = endindex-size
            testindex = np.arange(startindex, endindex)
            trainindex = np.concatenate((np.arange(0, startindex), np.arange(endindex, len(X))))
            yield trainindex, testindex

```
The class KFold returns the indices of the folds for the data. It is initalized with the number of splits, and then the function split is used to split a matrix. It only returns the indices, so the actual rows of the split have to be done separately. 

```python
def cv_score(model, X, y, cv, score_func):
    '''
    Returns the cross-validated
    :param model: the model to use
    :param X: the design matrix X
    :param y: the true values y
    :param cv: the cross-validation object to use, which must implement split
    :param score_func: the scoring function to use
    :return: the mean score of scores returned by score_func for each cross-validation split
    '''
    scores = list()
    for trainindex, testindex in cv.split(X):
        trainX = X[trainindex]
        trainY = y[trainindex]
        testX = X[testindex]
        testY = y[testindex]
        model.fit(trainX, trainY)
        predy = model.predict(testX)
        score = score_func(predy, testY)
        scores.append(score)
    return np.mean(scores)
```
The function cv_score does the work of calculating the cv-score for any data. It accepts any scoring function which returns a floating point value for prediction and true vector pairs. The function cv_accuracy_score and cv_squares_score are convenience methods which select the scoring method.


# Mushroom data set
The data set is the one described in http://archive.ics.uci.edu/ml/datasets/Mushroom . More formally the data set is called "Agaricus lepiota".

The data set contains attributes about different mushrooms, and a binary target attribute describing whether the mushroom is edible or not. 51,8% of the examples are edible, and 48,2% are not. The baseline accuracy There are 8124 instances and 22 categorical attributes. It was quickly discovered that this prediction task using all the data is very easy: using the entire data set one can acquire 100% accuracy.
Attributes were encoded using MultiLabelBinarizer to binary attributes.

## Results
For debugging purposes, the results were compared to scikit-learn implementation of KNN. Small fluctuations are possible because of tie-breaking situations.
For the entire data set, the cross-validated mean prediction accuracy is 100% for n_neighbors=1. Increasing the number of neighbours does not seem to lower the accuracy. By limiting the number of rows used, we can better compare the results. Using 100 rows and n_neighbors=2, the prediction accuracy is 0.95, which is exactly the same as Scikit-Learn's result. 

The following plots were made by using 200 rows of the original data set using both classification and regression scoring. As said previously, using the entire data set is not very interesting, because accuracy is almost always 100% or very close to it.

![](agaricus_neighbors_False.png)
![](agaricus_neighbors_True.png)

# Appendices

## knn.py
```python
import numpy as np
from cv import accuracy_score, squared_diff_score

class KNN:
    def __init__(self, n_neighbors=5, metric=None, regression=False):
        '''
        Initializes the KNN algorithm.
        :param n_neighbors: The number of neighbors to use.
        :param metric: The metric to use. Any distance measure is accepted, even non-metrics. The function must accept a
        1D vector as its first parameter, and a 2D matrix as the second parameter. It should calculate the rowwise distance
        between the first vector and all row vectors in the matrix.
        :param regression: Set to true for regression calculation. Defaults to False, which is classification.
        '''
        self.n_neighbors = n_neighbors
        if metric == None:
            metric = KNN.__euclidean
        self.metric = metric
        self.regression = regression
        self.classification = not regression
        if not regression:
            self.prediction_func = KNN.__classification_prediction
        else:
            self.prediction_func = KNN.__regression_prediction


    def fit(self, X, y):
        # Metrics typically can't handle ints.
        self.X = X.astype(np.float64)
        self.y = y


    def predict(self, X):
        '''
        Returns the KNN prediction for design matrix X.
        :param X: the design matrix X
        :return: the predicted values for rows of X
        '''
        dist = self.__get_distance_matrix(X)

        ranks = np.apply_along_axis(KNN.__rank, 1, dist)
        # Using np.nan to denote values that are not nearest neighbors
        indicators = np.where(ranks < self.n_neighbors, 1, np.nan)

        y = np.multiply(indicators, self.y)

        # Construct new matrix without nan values
        newshape = (np.shape(X)[0], self.n_neighbors)
        y = (np.reshape(y[~np.isnan(y)], newshape))

        prediction = np.apply_along_axis(self.prediction_func, 1, y)
        return prediction

    def score(self, X, y):
        '''
        Returns a score for design matrix X and true labels y. Predicted values for X are scored against true values y.
        :param X: The design matrix to predict
        :param y: The true values of y
        :return: If this is a classification task, returns prediction accuracy
                 If this is a regression task, returns the sum of the squared errors
        '''
        predy = self.predict(X)
        if self.classification:
            return accuracy_score(predy, y)
        else:
            return squared_diff_score(predy, y)

    def __apply_metric_old(self, y):
        '''
        Applies the metric to each row in self.X paired with y
        :param y: the y to give as parameter to metric
        :return: 1-dimensional distance vector, where each value is the pairwise distance of corresponding row in X and y
        '''
        metric = lambda x: self.metric(x, y)
        dist = np.apply_along_axis(metric, 1, self.X)
        return dist

    def __get_distance_matrix(self, X):
        X = X.astype(np.float64)
        # Bind self to apply_metric, needed to bind self.X
        apply_metric = lambda y : self.metric(y, self.X)
        dist = np.apply_along_axis(apply_metric, 1, X)
        return dist

    @staticmethod
    def __rank(x):
        '''
        Returns the ranks of elements of x. Ranks start from 0.
        :param x: 1D-vector to rank
        :return: The ranks of the elements of x.
        '''
        order = x.argsort()
        ranks = np.empty(len(x), int)
        ranks[order] = np.arange(len(x))
        return ranks


    @staticmethod
    def __euclidean(y, X):
        '''
        The euclidean distance metric
        :param y:
        :param X:
        :return:
        '''
        axis = np.ndim(X)-1
        return np.sqrt(np.sum((y-X)**2, axis=axis))

    @staticmethod
    def __classification_prediction(y):
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]

    @staticmethod
    def __regression_prediction(y):
        return np.mean(y)


```

## cv.py
```python
import numpy as np

class KFold:
    def __init__(self, n_splits=4):
        self.n_splits = n_splits

    def split(self, X):
        minfoldsize = len(X)//self.n_splits
        rem = len(X)-minfoldsize*self.n_splits #remainder, when the splits are not even
        foldsizes = np.repeat(minfoldsize, self.n_splits)
        foldsizes[:rem] += 1 #remainders are evenly added to first rem folds
        endindices = np.cumsum(foldsizes)

        for i, (size, endindex) in enumerate(zip(foldsizes, endindices)):
            startindex = endindex-size
            testindex = np.arange(startindex, endindex)
            trainindex = np.concatenate((np.arange(0, startindex), np.arange(endindex, len(X))))
            yield trainindex, testindex

def accuracy_score(predy, y):
    '''
    Returns the accuracy score for predicted y and true y
    :param predy: the predicted y
    :param y: the true y
    :return: the accuracy score
    '''
    return np.mean(np.where(np.isclose(predy, y), 1, 0))

def squared_diff_score(predy, y):
    '''
    Returns the sum of squared differences between predicted y and true y
    :param predy: predicted y
    :param y:
    :return:
    '''
    return np.sum((predy - y) ** 2)

def cv_score(model, X, y, cv, score_func):
    '''
    Returns the cross-validated
    :param model: the model to use
    :param X: the design matrix X
    :param y: the true values y
    :param cv: the cross-validation object to use, which must implement split
    :param score_func: the scoring function to use
    :return: the mean score of scores returned by score_func for each cross-validation split
    '''
    scores = list()
    for trainindex, testindex in cv.split(X):
        trainX = X[trainindex]
        trainY = y[trainindex]
        testX = X[testindex]
        testY = y[testindex]
        model.fit(trainX, trainY)
        predy = model.predict(testX)
        score = score_func(predy, testY)
        scores.append(score)
    return np.mean(scores)

def cv_accuracy_score(model, X, y, cv=KFold(n_splits=4)):
    '''
    Returns the cv-score using accuracy scoring.
    :see cv_score, accuracy_score
    '''
    return cv_score(model, X, y, cv=cv, score_func=accuracy_score)

def cv_squares_score(model, X, y, cv=KFold(n_splits=4)):
    '''
    Returns the cv-score using squared difference scoring
    :see cv_score, squared_diff_score
    '''
    return cv_score(model, X, y, cv=cv, score_func=squared_diff_score)
```

## agaricus_predict.py
```python
import numpy as np
import pandas as pd
from knn import KNN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier

from cv import KFold, cv_accuracy_score, cv_squares_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing.label import LabelBinarizer

import matplotlib.pyplot as plt

def get_agaricus_data(row_limit=None):
    df = pd.read_csv("data/agaricus-lepiota.data")
    # Randomize the rows because they are ordered
    df = df.sample(frac=1, random_state=1)
    X = df.values[:, 1:]

    y = df.values[:, 0]
    enc = [MultiLabelBinarizer() for _ in range(len(X[0]))]
    newX = np.empty((len(X),0), dtype=np.float64)
    for i, encoder in enumerate(enc):
        columns = encoder.fit_transform(X[:, i])
        newX = np.concatenate((newX, columns), axis=1)
    X = newX
    y = LabelBinarizer().fit_transform(y)
    y = y.flatten()

    if row_limit:
        X = X[:row_limit, :]
        y = y[:row_limit]
    return X,y

def test_knn(row_limit=None, n_neighbors=5):
    X, y = get_agaricus_data(row_limit=row_limit)
    models = (KNN(n_neighbors=n_neighbors), KNeighborsClassifier(n_neighbors=n_neighbors))
    names = ("Custom", "Scikit-learn")
    scores = []
    for model, name in zip(models, names):
        score = cv_accuracy_score(model, X, y)
        scores.append(score)
    diffs = []
    for score in scores:
        diffs.append(scores[0]-score)
    return list(zip(names, scores, diffs))

def test_multiple_neighbors(row_limit=None, regression=False, max_neighbors=10):
    X,y = get_agaricus_data(row_limit=row_limit)
    n_neighbors = np.arange(1,max_neighbors+1)
    score_func = cv_accuracy_score
    if regression:
        score_func = cv_squares_score
    scores = []
    for k in n_neighbors:
        model = KNN(n_neighbors=k, regression=False)

        score = score_func(model, X, y)
        scores.append(score)
    return (n_neighbors, scores)

def plot_neighbors(regression=False):
    plt.figure()
    n_neighbors, scores = test_multiple_neighbors(row_limit=200, regression=regression)
    plt.plot(n_neighbors,scores)
    ylabel = "accuracy score"
    type = "classification"
    if regression:
        ylabel = "squared difference"
        type = "regression"
    plt.suptitle("KNN CV mean vs. neighbors (%s)" % type)
    plt.xlabel("neighbors")
    plt.ylabel(ylabel)
    plt.savefig("agaricus_neighbors_" + str(regression) + ".png")


def help_print(scorelist):
    print("\n".join(["%s: %.5f (diff: %+.5f)" % (m, s, d) for m, s, d in scorelist]))

def plot_everything():
    plot_neighbors(regression=True)

    plot_neighbors(regression=False)
    print("For n=100 and n_neighbors=5")
    results = test_knn(row_limit=100, n_neighbors=5)
    help_print(results)

```